# Mechanisms of Action (MoA) Prediction Project: 
#### Data Source: https://www.kaggle.com/competitions/lish-moa/overview/description
#### Code Generated By: Mark D. Espinoza
-----

## Introduction

![picture 3](../images/21ad447f2d79344afabc719d92e5f9c98d6b025703213f3d3b8e65313116fbdf.png)

> Image 1: Women holding a drug with the appearance of analyzing it visually. 

Scientist are consistently working on creating new drug products, but the process of developing new drugs has significantly improved with the advent of new and more powerful technologies. Drug discovery has changed in that it is approached to target specific mechanism of a disease. Which is to identify a protein target associated with a disease and then develop a drug that is able to modulate the protein, thus reducing or eliminating the disease. 

Scientist describe the activity of a drug as mechanism-of-action, which entails the overall outcome the drug has on the targeted protein and other types of cells. 


### How Scientist determine the MOA of a drug? 

Treat human cells with a drug and the monitor and analyze the cellular responses with algorithms that search for similar patterns on large genomic databases. 

### Dataset Description

The dataset is a combination of gene expression and cell viability data. The data is from a new technology that measures simultaneously human cells' responses to drugs in a pool of 100 different cell types. Also the data has annotations for more than 5,000 drugs in that dataset. 

>Note: The data data is already split into training and testing subsets. Also, the goal is define the drug/molecule into one or more MOA classes ("multi-label classification problem")


### Objectives for this project

- The aim of this project is to develop and fit the best model for the predication of the best response to a specific target (gene expression and cell viability).

Other Questions/Insights from the data:
- What is the overall biological activity of a given molecule/drug? (Does it impact other cell type?)
- Are there any cell types that are always affected?
- Are there drugs that are intended for a specific cell type but have equal or greater effects on other cell types?


sources: 
- https://www.kaggle.com/competitions/lish-moa/overview/description

---
author:{Jin Paik, Maggie, mrbhbs, Steven Randazzo , tnat1031},
title:{Mechanisms of Action (MoA) Prediction},
publisher:{Kaggle},
year:{2020},
url:{https://kaggle.com/competitions/lish-moa}


---

## Literature Review:

![picture 4](../images/9461f0d4f4cfe03e5a435fd024a6aa9447ca6a1166d7f47f647e8d03479b37a0.png)

> Image 2: First - Publication regarding drug modeling: Multiscale modelling of drug mechanism and safety

Summary of the Article: 
A drug discovery program is setup to maximize the full knowledge of a molecule prior to every being tested/given to a human. There is a full understanding of a new drug in vitro and in vivo. There are many factors that are studied beyond just the MOA of a drug such as Negative Feedback regulation of a specific target. Additionally it is noted that drugs may have interactions with multiple targets, with varying affinities (affinity is related on how a molecule reaction/interaction/acceptance).
Modern drug discovery programs focus on targeting specific targets which have lead to the following examples of specific domains in drug development: 
1. Structure-based molecular modeling 
2. Ligand Based molecular modeling 
3. Modeling with omnics data


In summary drug development is increasingly becoming more multiscale modeling combining models from the initial determination of a target to the overall effect it has on a population. 




![picture 7](../images/67f626c53eca37b85b4af52e543b51287d938facd72e89086d12841e8c5d7a32.png)  

> Image 3: Second - Publication: Modeling drug mechanism of action with large scale gene‐expression profiles using GPAR, an artificial intelligence platform

Summary of the Article: 
With the advent of large drug-induced gene expression profiles machine learning/predictive modeling methods is an effective methods for discovering the MOA of drugs. However there is a lack of adoption by people since there is a lack in open source platforms/code and user friendly platforms. Overall they developed a GPAR method and online tool to connect MOA with gene expression signatures, providing a simple and effective deep learning-based modeling and prediction method for drug researchers. 

![picture 8](../images/f7e9c889247abf4cb83a93b9b86202e1256fc598800ca58976246ec01d28f5fc.png)  

> Image 4: Third - Publication: Mechanism-Based Pharmacodynamic Modeling

Summary of the Article: 
The modeling of drug development is promising and many of the modeling methods described in the article will be the start of more complex models. 
Types of Models Mentioned: 
- Simple Direct Effective Models: (Hill Equation: Represents a fundamental pharmacodynamic relationship)
- Biophase Distribution: In reference to plasma drug concentrations and time to reach target and the onset of side-effects
- Indirect Response Models: Reversible drug receptor interactions serve to alter the natural production or loss of biomarker response variables 
- and more... 

### Descriptive Analysis of the data

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import math
import time
from varclushi import VarClusHi
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss 
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection import IterativeStratification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.svm import SVC
import random
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from scipy import stats
import xgboost
from xgboost import XGBClassifier
%matplotlib inline

pd.options.display.max_columns = None

##### Import Data and view the dataframes


- train_features.csv - Features for the training set. Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low).
- train_drug.csv - This file contains an anonymous drug_id for the training set only.
- train_targets_scored.csv - The binary MoA targets that are scored.
- train_targets_nonscored.csv - Additional (optional) binary MoA responses for the training data. These are not predicted nor scored.
- test_features.csv - Features for the test data. You must predict the probability of each scored MoA for each row in the test data.

In [ ]:
#import both the data
train_features = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/train_features.csv")
test_features = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/test_features.csv")
test_features['flag']='test'
train_features['flag']='train'
train_drug = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/train_drug.csv")
train_targets_nonscored = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/train_targets_nonscored.csv")
train_targets_scored = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/train_targets_scored.csv")
f=train_features.merge(train_drug, how='left', on='sig_id')

merged=pd.concat([train_features,test_features], ignore_index=True, sort=False)

train_features_scored=pd.merge(train_features,train_targets_scored,how='inner')

#Datasets for treated and control experiments
treated= train_features[train_features['cp_type']=='trt_cp']
control= train_features[train_features['cp_type']=='ctl_vehicle']

#Datasets for treated and control: TEST SET
treated_t= test_features[test_features['cp_type']=='trt_cp']
control_t= test_features[test_features['cp_type']=='ctl_vehicle']

#Treatment time datasets
cp24= train_features[train_features['cp_time']== 24]
cp48= train_features[train_features['cp_time']== 48]
cp72= train_features[train_features['cp_time']== 72]

#Merge scored and nonscored labels
all_drugs= pd.merge(train_targets_scored, train_targets_nonscored, on='sig_id', how='inner')

#Treated drugs without control
treated_list = treated['sig_id'].to_list()
drugs_tr= train_targets_scored[train_targets_scored['sig_id'].isin(treated_list)]

#Select the columns c-
c_cols3 = [col for col in test_features.columns if 'c-' in col]
#Filter the TEST set
cells3=treated[c_cols3]


#view that all the data is there by seeing the shape 
print(train_features.shape)
print(test_features.shape)
print(train_drug.shape)
print(train_targets_nonscored.shape)
print(train_targets_scored.shape)

In [ ]:
train_features.head()

In [ ]:
test_features.head()

In [ ]:
train_drug.head()

In [ ]:
train_targets_nonscored.head()

In [ ]:
train_targets_scored.head()

##### Visualizations of training dataset

##### Treatment distribution

In [ ]:
treatment_group = train_features.groupby(['cp_type'])['sig_id'].count().reset_index()

treatment_group.columns = [
    'cp_type',  
    'count'
]

bar_plot_cp_type = px.bar(
    treatment_group, 
    x='cp_type', 
    y="count", 
    barmode='group',
    orientation='v', 
    title='cp_type',
    text_auto='.2s'
)
bar_plot_cp_type.update_layout(title_x=0.5)

bar_plot_cp_type.show()

> Distribution of treatment type - which is not balanced. Note there are few controls in contrast to treated. 

##### Dose distribution

In [ ]:
time_group = train_features.groupby(['cp_time'])['sig_id'].count().reset_index()

time_group.columns = [
    'cp_time', 
    'count'
]

time_plot = px.bar(
    time_group, 
    x='cp_time', 
    y="count",
    orientation='v',
    title='cp_time',
    text_auto='.2s'
)

time_plot.update_layout(title_x=0.5)

time_plot.show()

> Time distribution of when drugs were provided and it is distributed more evenly. 

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot( train_features['cp_time'], color='red', bins=5)
plt.title("Train: Treatment duration ", fontsize=15, weight='bold')
plt.show()

> Plot showing the time distribution of when drugs were given

##### Duration distribution

In [ ]:
dose_group = train_features.groupby(['cp_dose'])['sig_id'].count().reset_index()

dose_group.columns = [
    'cp_dose', 
    'count'
]

does_plot = px.bar(
    dose_group, 
    x='cp_dose', 
    y="count",
    orientation='v',
    title='cp_dose',
    text_auto='.2s'
)

does_plot.update_layout(title_x=0.5)

does_plot.show()

> The does distribution of low versus high even. 

#### Cell Viability
Cell Viability is defined as the number of healthy cells in a sample and proliferation of cells is a vital indicator for understanding the mechanisms in action of certain genes, proteins and pathways involved cell survival or death after exposing to toxic agents. 

sources: 
1. https://pubmed.ncbi.nlm.nih.gov/27604355/

In [ ]:
def plotf(f1, f2, f3, f4):
    plt.style.use('seaborn')
    sns.set_style('whitegrid')

    fig= plt.figure(figsize=(15,10))
    #2 rows 2 cols
    #first row, first col
    ax1 = plt.subplot2grid((2,2),(0,0))
    sns.distplot(train_features[f1], color='crimson')
    plt.title(f1,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    #first row sec col
    ax1 = plt.subplot2grid((2,2), (0, 1))
    sns.distplot(train_features[f2], color='gainsboro')
    plt.title(f2,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    #Second row first column
    ax1 = plt.subplot2grid((2,2), (1, 0))
    sns.distplot(train_features[f3], color='deepskyblue')
    plt.title(f3,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    #second row second column
    ax1 = plt.subplot2grid((2,2), (1, 1))
    sns.distplot(train_features[f4], color='black')
    plt.title(f4,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')

    return plt.show()


def plotd(f1):
    plt.style.use('seaborn')
    sns.set_style('whitegrid')
    fig = plt.figure(figsize=(15,5))
    #1 rows 2 cols
    #first row, first col
    ax1 = plt.subplot2grid((1,2),(0,0))
    plt.hist(control[f1], bins=4, color='mediumpurple',alpha=0.5)
    plt.title(f'control: {f1}',weight='bold', fontsize=18)
    #first row sec col
    ax1 = plt.subplot2grid((1,2),(0,1))
    plt.hist(treated[f1], bins=4, color='darkcyan',alpha=0.5)
    plt.title(f'Treated with drugs: {f1}',weight='bold', fontsize=18)
    plt.show()
    
def plott(f1):
    plt.style.use('seaborn')
    sns.set_style('whitegrid')
    fig = plt.figure(figsize=(15,5))
    #1 rows 2 cols
    #first row, first col
    ax1 = plt.subplot2grid((1,3),(0,0))
    plt.hist(cp24[f1], bins=3, color='deepskyblue',alpha=0.5)
    plt.title(f'Treatment duration 24h: {f1}',weight='bold', fontsize=14)
    #first row sec col
    ax1 = plt.subplot2grid((1,3),(0,1))
    plt.hist(cp48[f1], bins=3, color='lightgreen',alpha=0.5)
    plt.title(f'Treatment duration 48h: {f1}',weight='bold', fontsize=14)
    #first row 3rd column
    ax1 = plt.subplot2grid((1,3),(0,2))
    plt.hist(cp72[f1], bins=3, color='gold',alpha=0.5)
    plt.title(f'Treatment duration 72h: {f1}',weight='bold', fontsize=14)
    plt.show()


def ploth(data, w=15, h=9):
    plt.figure(figsize=(w,h))
    sns.heatmap(data.corr(), cmap='hot')
    plt.title('Correlation between targets', fontsize=25, weight='bold')
    return plt.show()

# corrs function: Show dataframe of high correlation between features
def corrs(data, col1='Gene 1', col2='Gene 2',rows=5,thresh=0.8, pos=[1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53]):
        #Correlation between genes
        corre= data.corr()
         #Unstack the dataframe
        s = corre.unstack()
        so = s.sort_values(kind="quicksort", ascending=False)
        #Create new dataframe
        so2= pd.DataFrame(so).reset_index()
        so2= so2.rename(columns={0: 'correlation', 'level_0':col1, 'level_1': col2})
        #Filter out the coef 1 correlation between the same drugs
        so2= so2[so2['correlation'] != 1]
        #Drop pair duplicates
        so2= so2.reset_index()
        pos = pos
        so3= so2.drop(so2.index[pos])
        so3= so3.drop('index', axis=1)
        #Show the first 10 high correlations
        cm = sns.light_palette("Red", as_cmap=True)
        s = so3.head(rows).style.background_gradient(cmap=cm)
        print(f"{len(so2[so2['correlation']>thresh])/2} {col1} pairs have +{thresh} correlation.")
        return s


def plotgene(data):
    sns.set_style('whitegrid')    
    data.plot.bar(color=sns.color_palette('Reds',885), edgecolor='black')
    set_size(13,5)
    #plt.xticks(rotation=90)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    plt.ylabel('Gene expression values', weight='bold')
    plt.title('Mean gene expression of the 772 genes', fontsize=15)
    return plt.show()

def mean(row):
    return row.mean()

def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)
    
def unidrug(data):
    #Filter out just the treated samples
    scored= data[data['sig_id'].isin(treated_list)]

    #Count unique values per column
    cols = data.columns.to_list() # specify the columns whose unique values you want here
    uniques = {col: data[col].nunique() for col in cols}
    uniques=pd.DataFrame(uniques, index=[0]).T
    uniques=uniques.rename(columns={0:'count'})
    uniques= uniques.drop('sig_id', axis=0)
    return uniques

def avgdrug(data):
    
    uniques=unidrug(data)

     #Calculate the mean values
    scored= data[data['sig_id'].isin(treated_list)]
    average=scored.mean()
    average=pd.DataFrame(average)
    average=average.rename(columns={ 0: 'mean'})
    average['percentage']= average['mean']*100
    
    return average

def avgfiltered(data):
    
    average= avgdrug(data)
    #Filter just the drugs with mean >0.01
    average_filtered= average[average['mean'] > 0.01]
    average_filtered= average_filtered.reset_index()
    average_filtered= average_filtered.rename(columns={'index': 'drug'})
    return average_filtered

def plotc(data, column, width=10, height=6, color=('silver', 'gold','lightgreen','skyblue','lightpink'), edgecolor='black'):
    
        fig, ax = plt.subplots(figsize=(width,height))
        title_cnt=data[column].value_counts()[:15].sort_values(ascending=True).reset_index()
        mn= ax.barh(title_cnt.iloc[:,0], title_cnt.iloc[:,1], color=sns.color_palette('Reds',len(title_cnt)))

        tightout= 0.008*max(title_cnt[column])
        ax.set_title(f'Count of {column}', fontsize=15, weight='bold' )
        ax.set_ylabel(f"{column}", weight='bold', fontsize=12)
        ax.set_xlabel('Count', weight='bold')
        if len(data[column].unique()) < 17:
            plt.xticks(rotation=65)
        else:
            plt.xticks(rotation=90)
        for i in ax.patches:
            ax.text(i.get_width()+ tightout, i.get_y()+0.1, str(round((i.get_width()), 2)),
             fontsize=10, fontweight='bold', color='grey')
        return
    
def plot_drugid(drug_id):
    g=d[f['drug_id']==drug_id]
    average_filtered2=avgfiltered(g)

    plt.figure(figsize=(5,2))
    average_filtered2.sort_values('percentage', inplace=True) 
    plt.scatter(average_filtered2['percentage'], average_filtered2['drug'], color=sns.color_palette('Reds',len(average_filtered2)))
    plt.title(f'Targets with higher presence in the drug: {drug_id} ', weight='bold', fontsize=15)
    plt.xticks(weight='bold')
    plt.yticks(weight='bold')
    plt.xlabel('Percentage', fontsize=13)
    return plt.show()


In [ ]:
#plot the cell-viability
plotf('c-10', 'c-18', 'c-72', 'c-95')

> Cell viability is graphed but note that is the value is more positive then that is an indication of a thriving/living cell, while negative means that the cells are dying/not well.
> If a cell is treated with a drug the expectation that the cell viability will decrease if it is the target cell.  

In [ ]:
plotd("c-75")

> Difference in control versus treated. As noted the treated should be more negative if it is the target cell for the new drug.

In [ ]:
plott("c-75")

> These plot show the overall effect of the treatment on a cell from 24 hr - 72 hr.

In [ ]:
#Select the columns c-
c_cols = [col for col in train_features.columns if 'c-' in col]
#Filter the columns c-
cells=treated[c_cols]
#Plot heatmap
plt.figure(figsize=(15,6))
sns.heatmap(cells.corr(), cmap='coolwarm', alpha=0.9)
plt.title('Correlation: Cell viability', fontsize=15, weight='bold')
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:
corrs(cells, 'Cell', 'Cell 2', rows=7)

#### Gene Expression
Gene expression is the process by which information from a gene is used in the synthesis of a functional gene product that enables it to produce end products, protein or non-coding RNA, and ultimately affect a phenotype, as the final effect.

sources: 
1. https://en.wikipedia.org/wiki/Gene_expression

In [ ]:
plotf('g-10','g-100','g-200','g-400')

> Gene Expression plot showing the distribution of gene expression. (Positive = expressed, Negative = Not Expressed)

In [ ]:
plotd('g-510')

> Treated versus not Treated expression. (Here is can be seen that the expression has increased for the gene) 


In [ ]:
plott('g-510')

> View the gene expression throughout time and seeing the impact for suppression at 48h. 

In [ ]:
#Select the columns g-
g_cols = [col for col in train_features.columns if 'g-' in col]
#Filter the columns g-
genes=treated[g_cols]
#Plot heatmap
# plt.figure(figsize=(15,7))
plt.figure(figsize=(15,6))
sns.heatmap(genes.corr(), cmap='coolwarm', alpha=0.9)
plt.title('Gene expression: Correlation', fontsize=15, weight='bold')
plt.show()

> Correlation of gene expression

In [ ]:
corrs(genes, 'Gene', 'Gene 2')

In [ ]:
#Correlation between drugs
corre= genes.corr()
#Unstack the dataframe
s = corre.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
#Create new dataframe
so2= pd.DataFrame(so).reset_index()
so2= so2.rename(columns={0: 'correlation', 'level_0':'Drug 1', 'level_1': 'Drug2'})
#Filter out the coef 1 correlation between the same drugs
so2= so2[so2['correlation'] != 1]
#Drop pair duplicates
so2= so2.reset_index()
so2= so2.sort_values(by=['correlation'])
pos = [1,3,5,7,9,11,13,15,17,19,21]
so2= so2.drop(so2.index[pos])
so2= so2.round(decimals=4)
so2=so2.drop('index', axis=1)
so3=so2.head(4)
#Show the first 10 high correlations
cm = sns.light_palette("Red", as_cmap=True)
s = so2.head().style.background_gradient(cmap=cm)
s

There are 34 genes that have a high pair correlation with other genes
It should also be noted that there are both positive and negative correlations - this implying that some drugs may be designed to increase the expression of a gene (upregulate/downregulate).

#### MOA

In [ ]:
average= avgdrug(train_targets_scored)
uniques=unidrug(train_targets_scored)
average_filtered=avgfiltered(train_targets_scored)

plt.style.use('seaborn')
sns.set_style('whitegrid')
fig = plt.figure(figsize=(15,5))
#1 rows 2 cols
#first row, first col
ax1 = plt.subplot2grid((1,2),(0,0))
sns.countplot(uniques['count'], color='deepskyblue', alpha=0.75)
plt.title('Unique elements per target [0,1]', fontsize=15, weight='bold')
#first row sec col
ax1 = plt.subplot2grid((1,2),(0,1))
sns.distplot(average['percentage'], color='orange', bins=20)
plt.title("The targets mean distribution", fontsize=15, weight='bold')
plt.show()

> All targets can be found in the samples

In [ ]:
plt.figure(figsize=(7,7))
average_filtered.sort_values('percentage', inplace=True) 
plt.scatter(average_filtered['percentage'], average_filtered['drug'], color=sns.color_palette('Reds',len(average_filtered)))
plt.title('Targets with higher presence in train samples', weight='bold', fontsize=15)
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.xlabel('Percentage', fontsize=13)
plt.show()

> Distribution of types of labels in the data

In [ ]:
inhibitors = [col for col in train_targets_scored.columns if 'inhibitor' in col]
activators = [col for col in train_targets_scored.columns if 'activator' in col]
antagonists = [col for col in train_targets_scored.columns if 'antagonist' in col]
agonists = [col for col in train_targets_scored.columns if 'agonist' in col]
modulators = [col for col in train_targets_scored.columns if 'modulator' in col]
receptors = [col for col in train_targets_scored.columns if 'receptor' in col]
receptors_ago = [col for col in train_targets_scored.columns if 'receptor_agonist' in col]
receptors_anta = [col for col in train_targets_scored.columns if 'receptor_antagonist' in col]


labelss= {'Drugs': ['inhibitors', 'activators', 'antagonists', 'agonists', 'receptors', 'receptors_ago', 'receptors_anta'],
          'Count':[112,5,32,60, 53, 24, 26]}


labels= pd.DataFrame(labelss)
labels=labels.sort_values(by=['Count'])
plt.figure(figsize=(15,5))
plt.bar(labels['Drugs'], labels['Count'], color=sns.color_palette('Reds',len(labels)))
plt.xticks(weight='bold')
plt.title('Target types', weight='bold', fontsize=15)
plt.show()

> The majority of targets are inhibitors

In [ ]:
ploth(drugs_tr)

> Correlation matrix: Showing there are very few strong correlations with targets.

In [ ]:
#Correlation between drugs
corre= drugs_tr.corr()
#Unstack the dataframe
s = corre.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
#Create new dataframe
so2= pd.DataFrame(so).reset_index()
so2= so2.rename(columns={0: 'correlation', 'level_0':'Target 1', 'level_1': 'Target 2'})
#Filter out the coef 1 correlation between the same drugs
so2= so2[so2['correlation'] != 1]
#Drop pair duplicates
so2= so2.reset_index()
pos = [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35]
so2= so2.drop(so2.index[pos])
so2= so2.round(decimals=4)
so2=so2.drop('index', axis=1)
so3=so2.head(4)
#Show the first 10 high correlations
cm = sns.light_palette("Red", as_cmap=True)
s = so2.head().style.background_gradient(cmap=cm)
s

> Targets with the highest MOA correlation

In [ ]:
plt.figure(figsize=(8,10))
the_table =plt.table(cellText=so3.values,colWidths = [0.35]*len(so3.columns),
          rowLabels=so3.index,
          colLabels=so3.columns
          ,cellLoc = 'center', rowLoc = 'center',
          loc='left', edges='closed', bbox=(1,0, 1, 1)
         ,rowColours=sns.color_palette('Reds',10))
the_table.auto_set_font_size(False)
the_table.set_fontsize(10.5)
the_table.scale(2, 2)
average_filtered.sort_values('percentage', inplace=True) 
plt.scatter(average_filtered['percentage'], average_filtered['drug'], color=sns.color_palette('Reds',len(average_filtered)))
plt.title('Targets with higher presence in train samples', weight='bold', fontsize=15)
plt.xlabel('Percentage', weight='bold')
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

> View the high correlation between targets

### Dimensions Reduction

Before running any models and training the data, we are going to clean and remove anything that is not important via Principal Component Analysis(PCA). [Dimensionality Reduction]

In [ ]:
all_data = pd.concat([train_features, test_features], ignore_index=True, sort=False)
all_data

In [ ]:
train_features_scored=pd.merge(train_features,train_targets_scored,how='inner')

In [ ]:
#remove any outliers
def cap_outliers(col):
    col[col>3]=3
    col[col<-3]=-3
    return col

In [ ]:
#Filtering all the numeric columns
numcols=all_data._get_numeric_data().columns
all_data_num=all_data.loc[:,numcols]
all_data_num=all_data_num.iloc[:,1:]

#z=np.abs(stats.zscore(all_data_num['g-0']))
#Calculate Z Scores for all the variables. 
all_data_num=all_data_num.apply(stats.zscore)

#Cap the outliers
all_data_num=all_data_num.apply(cap_outliers)


In [ ]:
# PCA function
def pca_application(df,n_components,pattern):
    df_p=df.loc[:, df.columns.str.startswith(pattern)]
    x = StandardScaler().fit_transform(df_p)
    pca = PCA(n_components=n_components)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents)
    return principalDf,pca

In [ ]:
#Calculate principal components separately for GE & CV columns
principalDf_g,pca_g=pca_application(all_data,200,'g-')
principalDf_c,pca_c=pca_application(all_data,30,'c-')

In [ ]:
z=np.arange(start=1, stop=len(pca_g.explained_variance_ratio_)+1, step=1)
z

plt.bar(x=z,height=pca_g.explained_variance_ratio_)
plt.xlabel('Principal Components')
plt.ylabel('Explained variance')
plt.title('Explained Variance for Gene Expression Variable PCAs')
plt.show()

plt.plot(np.cumsum(pca_g.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('Cumulative Explained Variance for Gene Expression Variable PCAs')

In [ ]:
z=np.arange(start=1, stop=len(pca_c.explained_variance_ratio_)+1, step=1)
z

plt.bar(x=z,height=pca_c.explained_variance_ratio_)
plt.xlabel('Principal Components')
plt.ylabel('Explained variance')
plt.title('Explained Variance for Cell Viability Variable PCAs')
plt.show()

plt.plot(np.cumsum(pca_c.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('Cumulative Explained Variance for Cell Viability Variable PCAs')

> These plots show the expanded variance plots and the purpose is to identify where the curve starts to flatten. Overall the cell viability have a high correlation with each other, while the gene expression do not. 

The aim is then to extract 115 Gene expression components and 15 cell viability. 

In [ ]:
#Extracting the principal components
pca_g=principalDf_g.iloc[:,:115]
l=[]
for i in range(1,116):
    var='pca_g'+str(i)
    l.append(var)
#l
pca_g.columns=l
pca_g

In [ ]:
#Extracting the principal components
pca_c=principalDf_c.iloc[:,:15]
#pca_c.columns=['pca_c1','pca_c2','pca_c3','pca_c4','pca_c5']
#pca_c
l=[]
for i in range(1,16):
    var='pca_c'+str(i)
    l.append(var)
#l
pca_c.columns=l
pca_c

#Merging the principal components dataframes
pca_cg=pd.merge(pca_c, pca_g, left_index=True, right_index=True)
pca_cg

In [ ]:
cp_cols=all_data.iloc[:,1:4]
pca_cg_cp=pd.merge(cp_cols, pca_cg, left_index=True, right_index=True)
pca_cg_cp['flag']=all_data['flag']

In [ ]:
#Calculating dummies for categorical variables
pca_cg_cp=pd.get_dummies(pca_cg_cp, columns=['cp_type', 'cp_dose'])
pca_cg_cp

### Create the model

In [ ]:
pca_cg_cp_train=pca_cg_cp.loc[pca_cg_cp['flag']=='train',:]
pca_cg_cp_train
del pca_cg_cp_train['flag']


In [ ]:
X=pca_cg_cp_train #Selecting feature variables
Y=train_features_scored.iloc[:,-207:-1] #Selecting the output columns
feature_list=pca_cg_cp_train.columns

#Split data into train and test datasets
X_train,X_test,Y_train,Y_test=train_test_split(X, Y,test_size=0.3,random_state=1)

def get_tail_label(df: pd.DataFrame, ql=[0.05, 1.]) -> list:
    """
    Find the underrepresented targets.
    Underrepresented targets are those which are observed less than the median occurance.
    Targets beyond a quantile limit are filtered.
    """
    irlbl = df.sum(axis=0)
    irlbl = irlbl[(irlbl > irlbl.quantile(ql[0])) & ((irlbl < irlbl.quantile(ql[1])))]  # Filtering
    irlbl = irlbl.max() / irlbl
    threshold_irlbl = irlbl.median()
    tail_label = irlbl[irlbl > threshold_irlbl].index.tolist()
    return tail_label

def get_minority_samples(X: pd.DataFrame, y: pd.DataFrame, ql=[0.05, 1.]):
    """
    return
    X_sub: pandas.DataFrame, the feature vector minority dataframe
    y_sub: pandas.DataFrame, the target vector minority dataframe
    """
    tail_labels = get_tail_label(y, ql=ql)
    index = y[y[tail_labels].apply(lambda x: (x == 1).any(), axis=1)].index.tolist()
    
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

def nearest_neighbour(X: pd.DataFrame, neigh) -> list:
    """
    Give index of 10 nearest neighbor of all the instance
    
    args
    X: np.array, array whose nearest neighbor has to find
    
    return
    indices: list of list, index of 5 NN of each element in X
    """
    nbs = NearestNeighbors(n_neighbors=neigh, metric='euclidean', algorithm='kd_tree').fit(X)
    euclidean, indices = nbs.kneighbors(X)
    return indices


def MLSMOTE(X, y, n_sample, neigh=5):
    """
    Give the augmented data using MLSMOTE algorithm
    
    args
    X: pandas.DataFrame, input vector DataFrame
    y: pandas.DataFrame, feature vector dataframe
    n_sample: int, number of newly generated sample
    
    return
    new_X: pandas.DataFrame, augmented feature vector data
    target: pandas.DataFrame, augmented target vector data
    """
    indices2 = nearest_neighbour(X, neigh=5)
    n = len(indices2)
    new_X = np.zeros((n_sample, X.shape[1]))
    target = np.zeros((n_sample, y.shape[1]))
    for i in range(n_sample):
        reference = random.randint(0, n-1)
        neighbor = random.choice(indices2[reference, 1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val > 0 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbor,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    return new_X, target


In [ ]:
X_train = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/X_train.csv")
Y_train = pd.read_csv("/Users/mdespinoza/Desktop/Github/data/Y_train.csv")

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=0,min_samples_split=10)
model.fit(X_train,Y_train)#Fitting the model 


: 

: 

In [ ]:
#Generating predictions from Random Fores Models
pred_rf=model.predict(X_test)
pred_rf_proba=model.predict_proba(X_test)

In [ ]:

feat_importances = pd.Series(model.feature_importances_, index=feature_list)
feat_importances=feat_importances.sort_values()
feat_importances.plot(kind='barh',figsize=(16,16))#Plotting feature importance


In [ ]:

print('Model Accuracy')
print(model.score(X_test,Y_test))

In [ ]:
log_model= MultiOutputClassifier(LogisticRegression(max_iter=1000, tol=0.1, C = 0.5,verbose=0,random_state = 42))

In [ ]:
#Helper Functions

def Extract(lst): 
    return [item[:,1] for item in lst] 

def calc_loss_df(pred_proba):
    out=Extract(pred_proba)
    arr=np.array(out)
    arr1=np.transpose(arr)
    l=[]
    col=[]
    testcols=Y_test.columns
    y=np.array(Y_test)
    
    for i in range(0,206):
        a=arr1[:,i].astype('float')
        b=y[:,i].astype('int')
        if np.sum(b)>0:
            l.append(log_loss(b,a))
            col.append(testcols[i])

    err=pd.DataFrame(
        {'cols': col,
         'log_loss': l
        })
    err=err.sort_values(by='log_loss',ascending=False)
    return err  

def pred_transform(preds):
    out=Extract(preds)
    arr=np.array(out)
    arr1=np.transpose(arr)
    return arr1

In [ ]:
pred_rf_proba_t=pred_transform(pred_rf_proba)

print(log_loss(np.array(Y_test),np.array(pred_rf_proba_t)))#Compute Log loss

In [ ]:
# Calculating Log Loss by Individual Output Column
#y=np.array(Y_test)

err=calc_loss_df(pred_rf_proba)

err_head=err.head(10)
err_tail=err.tail(10)

fig2 = px.histogram(err, x="log_loss",opacity=0.6, title='RF - Distribution of Log Loss values on test dataset')
fig2.show()

fig=px.bar(err_head,x='cols',y='log_loss',title='RF - Output Variables with Highest Log Loss')
fig.show()

fig1=px.bar(err_tail,x='cols',y='log_loss',title='RF - Output Variables with Lowest Log Loss')
fig1.show()

In [ ]:
log_model= MultiOutputClassifier(LogisticRegression(max_iter=1000, tol=0.1, C = 0.5,verbose=0,random_state = 42))
log_model.fit(X_train,Y_train)#Fitting the model 
#log_model.fit(X_res,y_res)
#Generating predictions
pred_log_proba=log_model.predict_proba(X_test)
pred_log_proba_t=pred_transform(pred_log_proba)

In [ ]:
# Using parameters from https://www.kaggle.com/fchmiel/xgboost-baseline-multilabel-classification
xgb = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

params = {'estimator__colsample_bytree': 0.6522,
          'estimator__gamma': 3.6975,
          'estimator__learning_rate': 0.0503,
          'estimator__max_delta_step': 2.0706,
          'estimator__max_depth': 10,
          'estimator__min_child_weight': 31.5800,
          'estimator__n_estimators': 166,
          'estimator__subsample': 0.8639
         }

xgb.set_params(**params)
xgb.fit(X_train,Y_train)

In [ ]:
pred_xg_proba = xgb.predict_proba(X_test)
df_ll_xg=calc_loss_df(pred_xg_proba)
pred_xg_proba_t=pred_transform(pred_xg_proba)

print(log_loss(np.array(Y_test),pred_xg_proba_t))

In [ ]:
df_ll_log=calc_loss_df(pred_log_proba)
#print('Log Loss for Logistic Regression',df_ll_log)

fig = go.Figure()
fig.add_trace(go.Histogram(x=err['log_loss'],name='Random Forest'))
fig.add_trace(go.Histogram(x=df_ll_log['log_loss'],name='Logistic Regression'))
fig.add_trace(go.Histogram(x=df_ll_xg['log_loss'],name='XGBoost'))

fig.update_layout(barmode='overlay',title='Comparison of Distribution of Log Loss values for models')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.6)

fig.show()

In [ ]:
loss_actions_r=pd.merge(left=depsum,right=err,how='inner',left_on='action',right_on='cols')
loss_actions_r.columns=['sum_actions','action','cols','log_loss_rf']

loss_actions_r_x=pd.merge(left=loss_actions_r,right=df_ll_xg,how='inner')
loss_actions_r_x.columns=['sum_actions','action','cols','log_loss_rf','log_loss_xg']
#loss_actions_r_x

loss_actions_r_x_l=pd.merge(left=loss_actions_r_x,right=df_ll_log,how='inner')
loss_actions_r_x_l.columns=['sum_actions','action','cols','log_loss_rf','log_loss_xg','log_loss_logistic']
loss_actions_r_x_l

fig=go.Figure()
fig.add_trace(go.Scatter(x=loss_actions_r_x_l['sum_actions'],
                        y=loss_actions_r_x_l['log_loss_rf'],mode='markers',name='Random Forest'))
fig.add_trace(go.Scatter(x=loss_actions_r_x_l['sum_actions'],
                        y=loss_actions_r_x_l['log_loss_xg'],mode='markers',name='XGBoost'))
fig.add_trace(go.Scatter(x=loss_actions_r_x_l['sum_actions'],
                        y=loss_actions_r_x_l['log_loss_logistic'],mode='markers',name='Logistic Regression'))

fig.update_layout(title='Model Performance - Sum of MoAs for Dependent Variable in Train dataset vs Log Loss',xaxis_title='Sum of MoAs Output Variable',yaxis_title='Log Loss value for Output Variable')

fig.show()

## Results:

## Discussion: 